In [1]:
from __future__ import print_function
from keras.models import Model

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)


### The Data

In [2]:
data_path = "/home/ubuntu/data/"

In [3]:
#List of training images

import os, shutil
import pandas as pd
dataset_path = data_path + "train/"
train_images = pd.DataFrame(columns=["Class", "Image", "Imagepath"])
for (folder, subs, files) in os.walk(dataset_path):
    for filename in files:
        label = folder.split("/")[-1]
        imagepath = os.path.join(folder, filename)
        train_images = train_images.append({"Class":label, 
                                                "Image":filename, 
                                                "Imagepath":imagepath}, 
                                               ignore_index=True)

In [4]:
train_images = train_images[train_images["Image"] != ".DS_Store"]

In [5]:
#Save train image list
train_images.to_csv(data_path+"train_set.csv", index=False)

In [6]:
#List of test images

dataset_path = data_path + "test_stg1/"
test_images = pd.DataFrame(columns=["Image", "Imagepath"])
for (folder, subs, files) in os.walk(dataset_path):
    for filename in files:
        imagepath = os.path.join(folder, filename)
        test_images = test_images.append({"Image":filename, 
                                                "Imagepath":imagepath}, 
                                               ignore_index=True)

In [7]:
test_images = test_images[test_images["Image"] != ".DS_Store"]

In [8]:
#Save test image list
test_images.to_csv(data_path+"test_set.csv", index=False)

In [9]:
#Image lists
train_image_list = train_images["Imagepath"].as_matrix()
test_image_list = test_images["Imagepath"].as_matrix()

print(len(train_image_list))
print(len(test_image_list))

3777
1000


### Feature Extraction

Extracting features with pretrained deep models

In [10]:
#Function to load an image as a numpy matrix

import numpy as np
from keras.preprocessing.image import img_to_array, load_img
def get_image_as_X(path_to_image_file, target_size, dim_ordering='th'):
    img = load_img(path_to_image_file, grayscale=False, target_size=target_size)
    x = img_to_array(img, dim_ordering=dim_ordering)
    x = np.expand_dims(x, axis=0)
    return x

In [11]:
#Function to extract features from an image with a pretrained model

def extract_features(image_list, image_size, model, preprocessing_function, dim_ordering='th'):
    features = []
    processed = []
    for image in image_list:
        try:
            x = get_image_as_X(image, image_size, dim_ordering)
            x = preprocessing_function(x)
            p = model.predict(x)
            features.extend(p)
            processed.append(image)
        except Exception as e:
            print("Fail with image:", image)
            print(e)
            continue
    
    return features, processed

In [12]:
from keras import backend as K
K.set_image_dim_ordering('th')

In [13]:
import warnings
warnings.filterwarnings("ignore")

Model: Inception V3, with weights pre-trained on ImageNet.

In [21]:
img_size=(299, 299)
from keras.applications import inception_v3
base_model = inception_v3.InceptionV3(weights='imagenet', include_top=True)
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 3, 299, 299)   0                                            
____________________________________________________________________________________________________
convolution2d_377 (Convolution2D)(None, 32, 149, 149)  896         input_5[0][0]                    
____________________________________________________________________________________________________
batchnormalization_377 (BatchNorm(None, 32, 149, 149)  64          convolution2d_377[0][0]          
____________________________________________________________________________________________________
convolution2d_378 (Convolution2D)(None, 32, 147, 147)  9248        batchnormalization_377[0][0]     
___________________________________________________________________________________________

In [ ]:
model = Model(input=base_model.input, output=base_model.get_layer('flatten').output)

In [35]:
%%time
features_train,_ = extract_features(train_image_list, img_size, model, inception_v3.preprocess_input)

CPU times: user 4min 26s, sys: 1min 21s, total: 5min 47s
Wall time: 5min 48s


In [44]:
features_train = np.array(features_train)
print(features_train.shape)

(3777, 2048)


In [45]:
%%time
features_test,_ = extract_features(test_image_list, img_size, model, inception_v3.preprocess_input)
features_test = np.array(features_test)
print(features_test.shape)

(1000, 2048)
CPU times: user 1min 12s, sys: 22.1 s, total: 1min 34s
Wall time: 1min 35s


In [ ]:
np.save(data_path+"features_inception_v3_train.npy", features_train)
np.save(data_path+"features_inception_v3_test.npy", features_test)

Model: VGG19 model, with weights pre-trained on ImageNet.

In [14]:
img_size=(224, 224)
from keras.applications import vgg19
base_model = vgg19.VGG19(weights='imagenet', include_top=True)
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 64, 224, 224)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 64, 224, 224)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 64, 112, 112)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [15]:
model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)

In [18]:
%%time
features_train,_ = extract_features(train_image_list, img_size, model, vgg19.preprocess_input)

CPU times: user 2min 48s, sys: 45.2 s, total: 3min 33s
Wall time: 3min 33s


In [21]:
features_train = np.array(features_train)
print(features_train.shape)

(3777, 4096)


In [22]:
%%time
features_test,_ = extract_features(test_image_list, img_size, model, vgg19.preprocess_input)
features_test = np.array(features_test)
print(features_test.shape)

(1000, 4096)
CPU times: user 45.4 s, sys: 12.4 s, total: 57.8 s
Wall time: 57.8 s


In [23]:
np.save(data_path+"features_vgg19_train.npy", features_train)
np.save(data_path+"features_vgg19_test.npy", features_test)

Model: ResNet50, with weights pre-trained on ImageNet.

In [14]:
img_size=(224, 224)
from keras.applications import resnet50
base_model = resnet50.ResNet50(weights='imagenet', include_top=True)
base_model.summary()

102588416/102853280 [============================>.] - ETA: 0s

In [15]:
model = Model(input=base_model.input, output=base_model.get_layer('flatten_1').output)

In [16]:
%%time
features_train,_ = extract_features(train_image_list, img_size, model, resnet50.preprocess_input)

CPU times: user 3min 35s, sys: 1min 50s, total: 5min 25s
Wall time: 6min 27s


In [17]:
features_train = np.array(features_train)
print(features_train.shape)

(3777, 2048)


In [18]:
%%time
features_test,_ = extract_features(test_image_list, img_size, model, resnet50.preprocess_input)
features_test = np.array(features_test)
print(features_test.shape)

(1000, 2048)
CPU times: user 53.2 s, sys: 28.5 s, total: 1min 21s
Wall time: 1min 21s


In [19]:
np.save(data_path+"features_resnet50_train.npy", features_train)
np.save(data_path+"features_resnet50_test.npy", features_test)

Model: Xception V1, with weights pre-trained on ImageNet.

In [ ]:
#Jupyter needs to be restarted after setting KERAS_BACKEND=tensorflow 
K.set_image_dim_ordering('tf')

In [ ]:
img_size=(299, 299)
from keras.applications import xception
base_model = xception.Xception(weights='imagenet', include_top=True)
base_model.summary()

In [ ]:
model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)

In [ ]:
%%time
features_train,_ = extract_features(train_image_list, img_size, model, xception.preprocess_input, 'tf')

In [ ]:
np.array(features_train).shape

In [ ]:
features_train = np.array(features_train)
print(features_train.shape)

In [ ]:
%%time
features_test,_ = extract_features(test_image_list, img_size, model, xception.preprocess_input, 'tf')
features_test = np.array(features_test)
print(features_test.shape)

In [ ]:
np.save(data_path+"features_resnet50_train.npy", features_train)
np.save(data_path+"features_resnet50_test.npy", features_test)